In [1]:
import torch
import torchvision
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torch.autograd import Variable
from torch import optim
import torch.nn as nn
import torch.nn.functional as F
import os
import cv2
import random
from PIL import Image
from tqdm import tqdm_notebook as tqdm
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
import time as time

In [ ]:
class ImageTransform():    
    def __init__(self, resize, mean, std):
        self.data_transform = {
            'train': transforms.Compose([
                transforms.RandomResizedCrop(resize, scale=(0.5, 1.0)),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize(mean, std)
            ]),
            'val': transforms.Compose([
                transforms.Resize(256),
                transforms.CenterCrop(resize),
                transforms.ToTensor(),
                transforms.Normalize(mean, std)
            ])
        }
        
    def __call__(self, img, phase):#Executed when the instance is called.
        return self.data_transform[phase](img)

In [ ]:
cat_directory = r'./Cat'
dog_directory = r'./Dog'

cat_images_filepaths = sorted([os.path.join(cat_directory, f) for f in os.listdir(cat_directory)])   
#Retrieving cat image data
#sorted - Returns the data as a sorted list.
#os.path.join - Used to combine a path and a filename, or to combine fragmented paths into one.
#In other words, it combines the file path and the image path.
dog_images_filepaths = sorted([os.path.join(dog_directory, f) for f in os.listdir(dog_directory)])
images_filepaths = [*cat_images_filepaths, *dog_images_filepaths]    
#두 이미지 파일 경로를 합침
correct_images_filepaths = [i for i in images_filepaths]  

random.seed    
random.shuffle(correct_images_filepaths)
#train_images_filepaths = correct_images_filepaths[:20000] #If you want to improve performance, try increasing the training dataset.
#val_images_filepaths = correct_images_filepaths[20000:-10] #The validation set should also be increased along with the training set.
train_images_filepaths = correct_images_filepaths[:400]    
val_images_filepaths = correct_images_filepaths[400:-10]  
test_images_filepaths = correct_images_filepaths[-10:]    
print(train_images_filepaths, len(val_images_filepaths), len(test_images_filepaths))

['./Cat\\cat.67.jpg', './Cat\\cat.225.jpg', './Dog\\dog.175.jpg', './Cat\\cat.239.jpg', './Cat\\cat.82.jpg', './Cat\\cat.247.jpg', './Dog\\dog.128.jpg', './Cat\\cat.167.jpg', './Dog\\dog.145.jpg', './Cat\\cat.236.jpg', './Dog\\dog.177.jpg', './Dog\\dog.22.jpg', './Cat\\cat.204.jpg', './Dog\\dog.60.jpg', './Dog\\dog.174.jpg', './Dog\\dog.138.jpg', './Cat\\cat.165.jpg', './Dog\\dog.89.jpg', './Cat\\cat.223.jpg', './Dog\\dog.131.jpg', './Dog\\dog.124.jpg', './Cat\\cat.138.jpg', './Cat\\cat.220.jpg', './Cat\\cat.212.jpg', './Dog\\dog.16.jpg', './Dog\\dog.151.jpg', './Dog\\dog.103.jpg', './Cat\\cat.87.jpg', './Dog\\dog.219.jpg', './Cat\\cat.235.jpg', './Dog\\dog.50.jpg', './Cat\\cat.76.jpg', './Cat\\cat.116.jpg', './Cat\\cat.227.jpg', './Cat\\cat.50.jpg', './Cat\\cat.3.jpg', './Dog\\dog.187.jpg', './Cat\\cat.208.jpg', './Dog\\dog.12.jpg', './Cat\\cat.48.jpg', './Dog\\dog.18.jpg', './Cat\\cat.72.jpg', './Cat\\cat.142.jpg', './Dog\\dog.119.jpg', './Cat\\cat.21.jpg', './Cat\\cat.215.jpg', './C

In [ ]:
size = 224
mean = (0.485, 0.456, 0.406)#Average and standard deviation for each RGB channel
std = (0.229, 0.224, 0.225)
batch_size = 32

In [5]:
class DogvsCatDataset(Dataset):    
    def __init__(self, file_list, transform=None, phase='train'):    
        self.file_list = file_list
        self.transform = transform
        self.phase = phase
        
    def __len__(self):
        return len(self.file_list)
    
    def __getitem__(self, idx):       
        img_path = self.file_list[idx]
        img = Image.open(img_path)        
        img_transformed = self.transform(img, self.phase)
        
        label = img_path.split('/')[-1].split('.')[0]
        if 'Dog' in label :
            label = 1
        elif 'Cat' in label:
            label = 0
        return img_transformed, label

In [6]:
train_dataset = DogvsCatDataset(train_images_filepaths, transform=ImageTransform(size, mean, std), phase='train')
val_dataset = DogvsCatDataset(val_images_filepaths, transform=ImageTransform(size, mean, std), phase='val')
test_dataset = DogvsCatDataset(val_images_filepaths, transform=ImageTransform(size, mean, std), phase='val')

index = 0
print(train_dataset.__getitem__(index)[0].size())
print(train_dataset.__getitem__(index)[1])

torch.Size([3, 224, 224])
0


In [7]:
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
dataloader_dict = {'train': train_dataloader, 'val': val_dataloader}

batch_iterator = iter(train_dataloader)
inputs, label = next(batch_iterator)
print(inputs.size())
print(label)

torch.Size([32, 3, 224, 224])
tensor([0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1,
        0, 0, 1, 1, 0, 1, 0, 0])


In [ ]:
class AlexNet(nn.Module):
    def __init__(self) -> None:
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),#(224+4-11)/4+1=56
            nn.ReLU(inplace=True),#To save memory, the input values ​​are not stored separately.
            nn.MaxPool2d(kernel_size=3, stride=2),#56-3/2+1=28
            nn.Conv2d(64, 192, kernel_size=5, padding=2),#28-5+4+1=28
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),#28-3/2+1=14
            nn.Conv2d(192, 384, kernel_size=3, padding=1),#14-3+2/1+1=14
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),#14
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),#14
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),#7
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))#This means that average pooling will be performed with an output size of 6x6.
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 2),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)#Flattening
        x = self.classifier(x)
        return x

In [9]:
model = AlexNet()
model.to(device)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [10]:
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss()

In [11]:
from torchsummary import summary
summary(model, input_size=(3, 256, 256))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 63, 63]          23,296
              ReLU-2           [-1, 64, 63, 63]               0
         MaxPool2d-3           [-1, 64, 31, 31]               0
            Conv2d-4          [-1, 192, 31, 31]         307,392
              ReLU-5          [-1, 192, 31, 31]               0
         MaxPool2d-6          [-1, 192, 15, 15]               0
            Conv2d-7          [-1, 384, 15, 15]         663,936
              ReLU-8          [-1, 384, 15, 15]               0
            Conv2d-9          [-1, 256, 15, 15]         884,992
             ReLU-10          [-1, 256, 15, 15]               0
           Conv2d-11          [-1, 256, 15, 15]         590,080
             ReLU-12          [-1, 256, 15, 15]               0
        MaxPool2d-13            [-1, 256, 7, 7]               0
AdaptiveAvgPool2d-14            [-1, 25

In [ ]:
def train_model(model, dataloader_dict, criterion, optimizer, num_epoch):    
    since = time.time()
    best_acc = 0.0
    
    for epoch in range(num_epoch):
        print('Epoch {}/{}'.format(epoch + 1, num_epoch))
        print('-'*20)
        
        for phase in ['train', 'val']:            
            if phase == 'train':
                model.train()
            else:
                model.eval()
                
            epoch_loss = 0.0
            epoch_corrects = 0
            
            for inputs, labels in tqdm(dataloader_dict[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):#Track computation history only during learning
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        
                    epoch_loss += loss.item() * inputs.size(0)
                    epoch_corrects += torch.sum(preds == labels.data)
                    
            epoch_loss = epoch_loss / len(dataloader_dict[phase].dataset)
            epoch_acc = epoch_corrects.double() / len(dataloader_dict[phase].dataset)
            
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
   
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    return model

In [13]:
num_epoch = 10
model = train_model(model, dataloader_dict, criterion, optimizer, num_epoch)

Epoch 1/10
--------------------


C:\Users\user\AppData\Local\Temp\ipykernel_35048\198646428.py:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for inputs, labels in tqdm(dataloader_dict[phase]):


  0%|          | 0/13 [00:00<?, ?it/s]

tensor([[0.0232, 0.0165],
        [0.0259, 0.0219],
        [0.0094, 0.0168],
        [0.0124, 0.0212],
        [0.0260, 0.0186],
        [0.0151, 0.0151],
        [0.0259, 0.0216],
        [0.0178, 0.0237],
        [0.0198, 0.0306],
        [0.0219, 0.0253],
        [0.0223, 0.0266],
        [0.0202, 0.0231],
        [0.0268, 0.0217],
        [0.0194, 0.0223],
        [0.0241, 0.0272],
        [0.0181, 0.0187],
        [0.0239, 0.0205],
        [0.0177, 0.0240],
        [0.0207, 0.0235],
        [0.0253, 0.0188],
        [0.0214, 0.0205],
        [0.0126, 0.0170],
        [0.0260, 0.0254],
        [0.0195, 0.0220],
        [0.0179, 0.0205],
        [0.0124, 0.0181],
        [0.0225, 0.0176],
        [0.0176, 0.0212],
        [0.0202, 0.0232],
        [0.0197, 0.0218],
        [0.0242, 0.0238],
        [0.0274, 0.0218]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[0.0193, 0.0217],
        [0.0218, 0.0183],
        [0.0258, 0.0202],
        [0.0218, 0.0083],
        [0.0159, 0.0

  0%|          | 0/3 [00:00<?, ?it/s]

...

tensor([[0.0237, 0.0215],
        [0.0199, 0.0238],
        [0.0235, 0.0227],
        [0.0245, 0.0221],
        [0.0220, 0.0218],
        [0.0232, 0.0214],
        [0.0248, 0.0211],
        [0.0216, 0.0207],
        [0.0232, 0.0217],
        [0.0191, 0.0226],
        [0.0242, 0.0214],
        [0.0229, 0.0215],
        [0.0235, 0.0219],
        [0.0236, 0.0210],
        [0.0225, 0.0208],
        [0.0230, 0.0229],
        [0.0220, 0.0251],
        [0.0232, 0.0221],
        [0.0222, 0.0216],
        [0.0230, 0.0228],
        [0.0212, 0.0228],
        [0.0221, 0.0228],
        [0.0223, 0.0209],
        [0.0231, 0.0222],
        [0.0241, 0.0231],
        [0.0249, 0.0218],
        [0.0242, 0.0195],
        [0.0230, 0.0215],
        [0.0254, 0.0214],
        [0.0243, 0.0211],
        [0.0213, 0.0235],
        [0.0246, 0.0210]], device='cuda:0')
tensor([[0.0246, 0.0193],
        [0.0194, 0.0269],
        [0.0242, 0.0204],
        [0.0237, 0.0240],
        [0.0220, 0.0214],
        [0.0213, 0.0

  0%|          | 0/13 [00:00<?, ?it/s]

tensor([[0.0125, 0.0212],
        [0.0246, 0.0191],
        [0.0209, 0.0199],
        [0.0179, 0.0204],
        [0.0264, 0.0221],
        [0.0203, 0.0231],
        [0.0143, 0.0174],
        [0.0166, 0.0234],
        [0.0237, 0.0173],
        [0.0326, 0.0189],
        [0.0260, 0.0192],
        [0.0235, 0.0142],
        [0.0280, 0.0198],
        [0.0298, 0.0295],
        [0.0256, 0.0163],
        [0.0150, 0.0200],
        [0.0149, 0.0242],
        [0.0153, 0.0201],
        [0.0141, 0.0117],
        [0.0271, 0.0192],
        [0.0203, 0.0116],
        [0.0217, 0.0212],
        [0.0268, 0.0243],
        [0.0124, 0.0196],
        [0.0225, 0.0188],
        [0.0222, 0.0215],
        [0.0224, 0.0322],
        [0.0214, 0.0272],
        [0.0191, 0.0272],
        [0.0175, 0.0173],
        [0.0195, 0.0209],
        [0.0194, 0.0239]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[0.0118, 0.0258],
        [0.0191, 0.0172],
        [0.0208, 0.0194],
        [0.0287, 0.0207],
        [0.0139, 0.0

  0%|          | 0/3 [00:00<?, ?it/s]

tensor([[0.0267, 0.0184],
        [0.0228, 0.0207],
        [0.0265, 0.0196],
        [0.0275, 0.0191],
        [0.0249, 0.0187],
        [0.0262, 0.0184],
        [0.0277, 0.0181],
        [0.0246, 0.0176],
        [0.0261, 0.0186],
        [0.0220, 0.0195],
        [0.0271, 0.0184],
        [0.0259, 0.0184],
        [0.0264, 0.0188],
        [0.0266, 0.0180],
        [0.0254, 0.0178],
        [0.0259, 0.0198],
        [0.0250, 0.0219],
        [0.0261, 0.0191],
        [0.0252, 0.0185],
        [0.0259, 0.0198],
        [0.0242, 0.0198],
        [0.0251, 0.0198],
        [0.0252, 0.0178],
        [0.0261, 0.0190],
        [0.0269, 0.0201],
        [0.0279, 0.0187],
        [0.0273, 0.0163],
        [0.0260, 0.0184],
        [0.0284, 0.0183],
        [0.0273, 0.0180],
        [0.0242, 0.0204],
        [0.0276, 0.0179]], device='cuda:0')
tensor([[0.0275, 0.0163],
        [0.0223, 0.0237],
        [0.0271, 0.0173],
        [0.0266, 0.0209],
        [0.0249, 0.0182],
        [0.0242, 0.0